In [166]:
import pandas as pd
import numpy as np
import calendar

In [167]:
df = pd.read_csv('data/sp500_index.csv')
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date").sort_index()

In [168]:
print(df.head())
print(df.dtypes)

             S&P500
Date               
2014-12-22  2078.54
2014-12-23  2082.17
2014-12-24  2081.88
2014-12-26  2088.77
2014-12-29  2090.57
S&P500    float64
dtype: object


In [169]:
# Calculate daily returns
df["Return"] = df["S&P500"].pct_change()

In [171]:
# 1. Construct monthly features
monthly = pd.DataFrame()
monthly["Month_start"] = df["S&P500"].resample("ME").first()
monthly["Month_end"]   = df["S&P500"].resample("ME").last()
monthly["High"]        = df["S&P500"].resample("ME").max()
monthly["Low"]         = df["S&P500"].resample("ME").min()
monthly["Year"] = monthly.index.year

In [172]:
# Previous month return
monthly["Return_prev_month"] = (monthly["Month_end"] - monthly["Month_start"]) / monthly["Month_start"]

In [173]:
# Previous month volatility (based on daily returns)
monthly["Volatility_prev_month"] = df["Return"].resample("ME").std()

In [174]:
# Previous month high-low range
monthly["Range_prev_month"] = monthly["High"] - monthly["Low"]

In [175]:
# 2. Moving average features ===
# Daily moving averages (5-day, 20-day), take the last value of each month
df["MA5d"] = df["S&P500"].rolling(5).mean()
df["MA20d"] = df["S&P500"].rolling(20).mean()
monthly["MA5d_minus_MA20d"] = (df["MA5d"] - df["MA20d"]).resample("ME").last()

# Monthly moving averages (3 months, 6 months)
monthly["MA3m"] = monthly["Month_end"].rolling(3).mean()
monthly["MA6m"] = monthly["Month_end"].rolling(6).mean()
monthly["MA3m_minus_MA6m"] = monthly["MA3m"] - monthly["MA6m"]

# 3. Time Features
monthly["Year"] = monthly.index.year
monthly["Month"] = monthly.index.month
monthly["Month"] = monthly.index.month.map(lambda x: calendar.month_name[x])
dummies = pd.get_dummies(monthly["Month"])
monthly = pd.concat([monthly, dummies], axis=1).drop(columns=["Month"])

# 4. Label: Next month's return
monthly["Label_next_month"] = monthly["Return_prev_month"].shift(-1)

# Drop the last month (no label available)
monthly = monthly.dropna()

print(monthly.head())

            Month_start  Month_end     High      Low  Year  Return_prev_month  \
Date                                                                            
2015-05-31      2108.29    2107.39  2130.82  2080.15  2015          -0.000427   
2015-06-30      2111.73    2063.11  2124.20  2057.64  2015          -0.023024   
2015-07-31      2077.42    2103.84  2128.28  2046.68  2015           0.012718   
2015-08-31      2098.04    1972.18  2104.18  1867.61  2015          -0.059989   
2015-09-30      1913.85    1920.03  1995.31  1881.77  2015           0.003229   

            Volatility_prev_month  Range_prev_month  MA5d_minus_MA20d  \
Date                                                                    
2015-05-31               0.006810             50.67            4.4405   
2015-06-30               0.007012             66.56          -11.5195   
2015-07-31               0.007372             81.60            0.5380   
2015-08-31               0.016831            236.57          -85.59

In [176]:
print(monthly.describe())

       Month_start    Month_end         High          Low         Year  \
count   115.000000   115.000000   115.000000   115.000000   115.000000   
mean   3359.621130  3388.974870  3466.601304  3267.279043  2019.617391   
std    1040.994727  1064.993842  1082.298827  1019.182374     2.789651   
min    1913.850000  1920.030000  1951.700000  1829.080000  2015.000000   
25%    2476.450000  2513.105000  2550.215000  2422.465000  2017.000000   
50%    3066.910000  3044.310000  3153.630000  2964.330000  2020.000000   
75%    4197.350000  4187.485000  4302.685000  4090.205000  2022.000000   
max    5728.800000  6032.380000  6032.380000  5712.690000  2024.000000   

       Return_prev_month  Volatility_prev_month  Range_prev_month  \
count         115.000000             115.000000        115.000000   
mean            0.008778               0.009386        199.322261   
std             0.046372               0.006436        137.661515   
min            -0.163625               0.002918         3

In [177]:
monthly.to_csv("monthly_features.csv", index=True)